In [1]:
# Resolve path when used in a usecase project
import sys
from pathlib import Path

sys.path.insert(0, str(Path("../../").resolve()))

In [2]:
import logging

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [3]:
import recommend
print(f'Using {recommend.__version__} version of recommend package')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Using 0.33.0 version of recommend package


# `recommend.common_constraints` tutorial

## `create_similar_value_penalty`

Imagine you want to control 4 air flows as it it was a single parameter. I.e. all those four can be changed to a same value. This constraint can be introduces via `create_similar_value_penalty`.

This function creates `n-1` penalties which align those controls, where `n` is number of controls (read implementation details below). **TL;DR:** we take a first column and align all columns against it.

In [4]:
from recommend.common_constraints import create_similar_value_penalty

help(create_similar_value_penalty)

Help on function create_similar_value_penalty in module recommend.common_constraints.common_penlaties.common_penalties:

create_similar_value_penalty(controls: List[str], penalty_multiplier: float = 1.0) -> List[optimizer.constraint.penalty.Penalty]
    Creates constraint that aligns two or more controls'.
    
    I.e. creates such penalties that make sure ``controls``
    have similar recommended values.
    
    Implementation details:
        First control from ``controls`` is considered a reference control.
        All other controls are aligned against reference control.
        Hence, number of penalties is ``len(controls)-1``
        and each penalty is expressed as control "==" reference control.
    
    Example:
        Imagine three controls "A", "B", "C". Then we produce two penalties:
        "A" == "B" and "A" == "C".



Let's check out how those penalties behave.

In [5]:
air_flows = [
    "air_flow01",
    "air_flow02",
    "air_flow03",
    "air_flow04",
]
penalties = create_similar_value_penalty(controls=air_flows)
penalties

Now let's imagine we have three proposed sets of controls. And in first set we have all controls equal to the same. In the second set all control have set point 100 and only air_flow02 has 120. And in the third one all controls are different: 80/100/120/140.

In [6]:
import pandas as pd

proposed_controls = pd.DataFrame(
    [
        [100, 100, 100, 100],
        [100, 120, 100, 100],
        [80, 100, 120, 140],
    ],
    columns=air_flows,
)
proposed_controls

,air_flow01,air_flow02,air_flow03,air_flow04
0,100,100,100,100
1,100,120,100,100
2,80,100,120,140


Now let's evaluate our total penalty value as it does OptimizationProblem – by applying penalties one by one.

Note that we don't account for problem sense (max/min) and just want to calculate abs penalty value. The sign handling happens on the problem side

In [7]:
import numpy as np

total_penalty = np.zeros(len(proposed_controls))
for penalty in penalties:
    total_penalty += penalty(proposed_controls)
total_penalty

0      0.0
1     20.0
2    120.0
dtype: float64

So we can see that for the first case we have zero penalty, for the second 20 (only second control has this difference with the rest), and the final one has the total penalty of the 120.

Those differnces can be scaled by `penalty_multiplier` argument of `create_similar_value_penalty`.

## `create_similar_delta_penalty`

Imagine you want to control 4 air flows delta. And you can can only control four at once disregarding their inital values. Meaning that initial amount of air in all three might be slightly different, but the amount you add/subtract is equally divided among four of those.

So the recommendations you give for those controls must have the same delta wrt to initial controls' values.

We ca constraint the solutions space by adding penalties for unequal deltas. This can be done via `create_similar_delta_penalty`. This function creates `n-1` penalties which align  controls' deltas, where `n` is number of controls (read implementation details below). **TL;DR:** we take a first column's delta and align all columns' deltas against it.

In [8]:
from recommend.common_constraints import create_similar_delta_penalty

help(create_similar_delta_penalty)

Help on function create_similar_delta_penalty in module recommend.common_constraints.common_penlaties.common_penalties:

create_similar_delta_penalty(row_to_optimise: pandas.core.frame.DataFrame, controls: List[str], penalty_multiplier: float = 1.0) -> List[optimizer.constraint.penalty.Penalty]
    Creates constraint that aligns two or more controls' deltas.
    
    I.e. creates such penalties that make sure
    ``controls`` have similar recommended delta = initial value - recommended value.
    
    Implementation details:
        First control from ``controls`` is considered a reference control.
        All deltas are aligned against reference control's delta.
        Hence, number of penalties is ``len(controls)-1`` and each penalty
        is expressed as control delta "==" reference control delta.
    
    Example:
        Imagine three controls "A", "B", "C". Then we produce two penalties:
        delta("A") == delta("B") and delta("A") == delta("C").



Let's check out how those penalties behave. By definition to create those, we need initial control values. Let's create the single-row dataframe (often referred to as `row_to_optimize` in `recommend` package) with those.

In [9]:
import pandas as pd

initial_controls = pd.DataFrame(
    [[95, 100, 80, 85]],
    columns=air_flows,
)
initial_controls

,air_flow01,air_flow02,air_flow03,air_flow04
0,95,100,80,85


In [10]:
air_flows = [
    "air_flow01",
    "air_flow02",
    "air_flow03",
    "air_flow04",
]
penalties = create_similar_delta_penalty(
    row_to_optimise=initial_controls, controls=air_flows,
)
penalties

Now let's imagine we have three proposed sets of controls. In first set we have all controls' deltas the same, equal to 10. In the second set all control's deltas are equal to 10 and only air_flow02 has change of 20. And in the third one all controls's deltas are different: 10, 20, 30, and 40.

In [11]:
import pandas as pd

proposed_controls = pd.concat(
    [initial_controls, initial_controls, initial_controls],
).reset_index(drop=True)
deltas = pd.DataFrame(
    [
        [10, 10, 10, 10],
        [10, 20, 10, 10],
        [10, 20, 30, 40],
    ],
    columns=air_flows,
)
proposed_controls += deltas
proposed_controls

,air_flow01,air_flow02,air_flow03,air_flow04
0,105,110,90,95
1,105,120,90,95
2,105,120,110,125


Now let's evaluate our total penalty value as it does OptimizationProblem – by applying penalties one by one.

Note that we don't account for problem sense (max/min) and just want to calculate abs penalty value. The sign handling happens on the problem side

In [12]:
import numpy as np

total_penalty = np.zeros(len(proposed_controls))
for penalty in penalties:
    total_penalty += penalty(proposed_controls)
total_penalty

0     0.0
1    10.0
2    60.0
dtype: float64

So we can see that for the first case we have zero penalty, for the second 10 (this is the difference of the between second control's delta and the rest), and the final set has the total penalty of the 60.

Those differnces can be scaled by `penalty_multiplier` argument of `create_similar_delta_penalty`.

## `create_discrete_grid_repair`

Imagine you have a continuous solver and yet you'd like to have some parameter discrete (we don't recommend doing this as it solws down the optimizer and the preffered way of work is to choose mixed domain solver).

This can be done via repair created by `create_discrete_grid_repair`.

In [13]:
from recommend.common_constraints import create_discrete_grid_repair

help(create_discrete_grid_repair)

Help on function create_discrete_grid_repair in module recommend.common_constraints.common_repairs.discrete_grid_repair:

create_discrete_grid_repair(column: str, op_min: float, op_max: float, step_size: float) -> optimizer.constraint.repair.SetRepair
    Creates a new set-based constraint set repair for a given ``column``;
    this repair will map column to a discrete grid [op_min, op_max]
    with a fixed ``step_size``.
    
    Notes:
        Grid is created via ``_get_linear_search_space`` with following rules:
        * ``op_min`` and ``op_max`` is always included.
        * the step size from ``op_max`` to previous value might differ from
            ``step_size``
    
        Grid examples::
            >>> _get_linear_search_space(2.0, 4.0, 0.5)
            [2.0, 2.5, 3.0, 3.5, 4]
            >>> _get_linear_search_space(2.0, 4.0, 0.7)
            [2.0, 2.7, 3.4, 4.0]
            >>> _get_linear_search_space(2.0, 4.0, 5.0)
            [2.0, 4.0]



In this example we'll constrain the `ore_pulp_density` value to be discrete from 50 to 100 with a step size of 10.

This function creates a repair the control we pass with a given restrictions on the grid,  let'stry it out.

In [14]:
grid_repair = create_discrete_grid_repair(
    "ore_pulp_ph", op_min=50, op_max=100, step_size=10,
)
grid_repair

Now let's create a row with the ph level that is lower than lower bound, higher than upper bound, and in between the grid: closer to the left edge and to the right edge.

In [15]:
proposed_controls = pd.DataFrame(
    {"ore_pulp_ph": [40, 110, 53, 57]},
)
proposed_controls

,ore_pulp_ph
0,40
1,110
2,53
3,57


You can see the repair works as intended fixing out of bounds values to bounds (first and second row), and out of the grid to the closes grid value 53 -> 55, and 57 -> 60.

In [16]:
grid_repair(proposed_controls)

,ore_pulp_ph
0,50
1,100
2,50
3,60


## `create_repairs_for_discrete_grid_controls`

The same (refer to the `create_discrete_grid_repair` section) can be done to the set of controls. To do that you have to provide `ControlsConfig` (with configs for lower/upper bounds and step size) and names of the controls you'd like to constraint.

In [17]:
from recommend.common_constraints import create_repairs_for_discrete_grid_controls
help(create_repairs_for_discrete_grid_controls)

Help on function create_repairs_for_discrete_grid_controls in module recommend.common_constraints.common_repairs.discrete_grid_repair:

create_repairs_for_discrete_grid_controls(controls: List[str], controlled_parameters_config: recommend.controlled_parameters.config.ControlledParametersConfig) -> List[optimizer.constraint.repair.SetRepair]
    Generate repair for each control in ``controls`` that will map ``control``
    to a discrete grid using ``controlled_parameters_config``.
    
    Note: the repair will be returned only for those controlled parameters
    that have notnull step size in ``controlled_parameters_config``.
    
    Returns:
        list of repairs, one for reach control
        that has a specified step size in its config



In [18]:
from recommend import ControlledParametersConfig

controls_config = ControlledParametersConfig([
    {
        "name": "ore_pulp_ph",
        "op_min": 10,
        "op_max": 100,
        "step_size": 30,
    },
    {
        "name": "ore_pulp_density",
        "op_min": 80,
        "op_max": 90,
        "step_size": 2,
    },
])
create_repairs_for_discrete_grid_controls(
    ["ore_pulp_ph", "ore_pulp_density"], controls_config,
)